In [1]:
import sys
import os
import time
import torch
import pandas as pd
import traceback
import argparse

# If running in Jupyter notebook, remove unrecognized arguments
if '__file__' not in globals():  # Check if running in Jupyter
    sys.argv = sys.argv[:1]  # Remove the '-f' argument

sys.path.append('FLTLf')
from FLTLf.parser import LTLfParser
from FLTLf.converter import Converter
from FLTLf import core

# Argument parser for selecting device
parser = argparse.ArgumentParser()
parser.add_argument('-d', "--device", default="cpu")

args = parser.parse_args()
core.device = args.device
core.debug = False

# Set random seed for reproducibility
torch.manual_seed(0)

# Adjusted parameters for memory optimization
batch_sizes = [100, 1_000, 5_000]  # Reduce batch sizes
maxlengths = [500, 1_000]          # Shorten sequence length
num_AP = 20                        # Lower the number of predicates
predicate_names = [f"p{i}" for i in range(num_AP)]
number_of_formulas = 100

simple_declare_formulas = []
declare_models = []

# Loading all DECLARE formulas
with open(os.path.join("data", f"{num_AP}_AP_{number_of_formulas}_constraints.csv"), 'r') as file:
    for line in file:
        formula = line.strip().split(",")[1]
        if formula != 'formula':
            simple_declare_formulas.append(formula)

# Aggregation with ANDs
models_lengths = [25, 50, 75, 100]
for num_constraints in models_lengths:
    declare_model = "(" + ") & (".join(simple_declare_formulas[:num_constraints]) + ")"
    declare_models.append(declare_model)

results_per_formula = {formula: [] for formula in declare_models}

# Verbose printouts
verbose = False
savefile = True
skippadding = True

# Results tracking
num_runs = 10
results = {"Run": [], "Num. Cases": [], "Case Length": []}
batch_sizes_column = []
maxlengths_column = []
run_column = []

# Main loop for processing
for batch_size in batch_sizes:
    for maxlength in maxlengths:
        run_column.extend([i for i in range(num_runs)])
        batch_sizes_column.extend([batch_size] * num_runs)
        maxlengths_column.extend([maxlength] * num_runs)

        print("------times-------")
        start = time.time()

        # Generate a random tensor log
        tensor_log = torch.rand(batch_size, maxlength, len(predicate_names), dtype=torch.float32)
        rand = time.time()
        print(f"RANDOM GEN time {rand - start}")

        # Padding
        converter = Converter(predicate_names)
        tensor_log = converter.log2tensor(tensor_log, verbose, skippadding)

        if not skippadding:
            print(f"PADDING time {time.time() - rand}")
        else:
            print("PADDING skipped")

# Parsing into a formula
parser_instance = LTLfParser()

# Process each DECLARE model
for formula in declare_models:
    start = time.time()

    # Slice predicates not in the formula
    core.tensor_log, converter.predicate_names = converter.slice_tensor_log(tensor_log, formula, verbose)
    core.tensor_log = core.tensor_log.to(core.device)

    # Setting dimensions for practical access
    core.batch_size = converter.batch_size
    core.maxlength = converter.maxlength

    ready = time.time()
    print(f"SLICING TIME {ready - start}")

    try:
        pyformula = parser_instance(formula)
    except Exception as e:
        print(f"Error parsing formula '{formula}': {traceback.format_exc()}")

    print(f"PARSING TIME {time.time() - ready}")

    for run in range(num_runs):
        start = time.time()

        visitor = core.Visitor()
        print("------------------")
        print(f"Evaluation of {pyformula.print()} at instant {0}:")

        try:
            visitor.visit(pyformula, 0)
        except Exception as e:
            print(f"Error evaluating formula '{formula}' at run {run}: {traceback.format_exc()}")

        exec_time = time.time() - start
        if savefile:
            results_per_formula[formula].append(exec_time)

        print(f"({run}, {core.batch_size}, {core.maxlength}) -> {exec_time}")

# Save results
results["Run"] = run_column
results["Num. Cases"] = batch_sizes_column
results["Case Length"] = maxlengths_column

results_df = pd.DataFrame.from_dict(results_per_formula)
settings_df = pd.DataFrame.from_dict(results)
results_df = pd.concat([settings_df, results_df], axis=1)
results_df.rename(columns={
    declare_models[0]: "25 constraints",
    declare_models[1]: "50 constraints",
    declare_models[2]: "75 constraints",
    declare_models[3]: "100 constraints"
}, inplace=True)

if savefile:
    os.makedirs('results', exist_ok=True)
    results_df.to_csv(os.path.join('results', f'DECLARE_conformance_check_times_{core.device.upper()}.csv'))
    print("Results saved successfully.")


ModuleNotFoundError: No module named 'FLTLf'